In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/대학원 개인/NewB-master')
os.getcwd()

'/content/drive/MyDrive/대학원 개인/NewB-master'

In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split

conservative_data = pd.read_csv('conservative.txt', sep='\t', header=None).dropna()
liberal_data = pd.read_csv('liberal.txt', sep='\t', header=None).dropna()

# column 이름 변경 
conservative_data.rename(columns = {0: 'LABEL', 1 : 'REVIEW'}, inplace = True)
liberal_data.rename(columns = {0: 'LABEL', 1 : 'REVIEW'}, inplace = True)
liberal_data['LABEL'] = 'liberal' # label 구분
conservative_data['LABEL'] = 'conservative' # label 구분

# liberal_data.head()
total_data = pd.concat([conservative_data, liberal_data])

# csv 파일로 저장
X_train, X_test, y_train, y_test = train_test_split(total_data['REVIEW'], total_data['LABEL'],test_size=0.3,
                                                    stratify=total_data['LABEL'], random_state=0)

train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

train_data.to_csv('data/train_data.csv', index=False)
test_data.to_csv('data/test_data.csv', index=False)

In [3]:
!pip install nltk

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [1]:
# 필요한 library imort 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
# from torchtext.legacy import data 
# from torchtext import datasets
import random
import numpy as np
from nltk import word_tokenize

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
SEED = 123
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

print(device)

cpu


In [46]:
KERNEL_SIZE = [3, 4, 5] # 총 3개의 kernel size 사용 (KERNEL_SIZE, embed_dimension)
BASE_MODEL = 'CNN'

def tokenizer(text):
    if BASE_MODEL == 'CNN':
        token = word_tokenize(text)
        if len(token) < max(KERNEL_SIZE): 
            for i in range(0, max(KERNEL_SIZE)-len(token)):
                token.append('<PAD>') # 커널 사이즈 보다 문장의 길이가 작은 경우 에러 방지
    else:
        token = word_tokenize(text)
    return token

# 필드 정의
# 배치 우선 여부(True일 경우 텐서 크기의 0번째 인덱스는 배치사이즈로 설정)
REVIEW = data.Field(tokenize=tokenizer, batch_first = True) # 배치 우선 여부(True일 경우 텐서 크기의 0번째 인덱스는 배치사이즈로 설정)
LABEL = data.LabelField(dtype=torch.float)

# {csv컬럼명 : (데이터 컬럼명, Field이름)} / id는 사용 x
fields = {'REVIEW': ('REVIEW', REVIEW), 'LABEL': ('LABEL', LABEL)}

train_data, test_data = data.TabularDataset.splits(path = 'data', # 데이터 파일 경로 
                                                   train = 'train_data.csv',
                                                   test = 'test_data.csv',
                                                   format = 'csv', # 데이터 파일 형식
                                                   fields = fields)

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [37]:
# 결과 확인
print('훈련 샘플의 개수 : {}'.format(len(train_data)))
print('테스트 샘플의 개수 : {}'.format(len(test_data)))
print(vars(train_data[3]))

훈련 샘플의 개수 : 113048
테스트 샘플의 개수 : 69213
{'REVIEW': ['but', 'they', 'have', 'stood', 'by', 'the', 'notion', 'that', 'trump', 'will', 'not', 'be', 'bound', 'by', 'traditional', 'rules'], 'LABEL': 'conservative'}


In [48]:
# 단어 집합 만들기
MAX_VOCAB_SIZE = 50000 # 단어 집합의 최대 크기

REVIEW.build_vocab(train_data,
                   max_size = MAX_VOCAB_SIZE,
                   vectors = 'fasttext.simple.300d',
                   unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_data)

print('단어 집합의 크기 : {}'.format(len(REVIEW.vocab)))
print(REVIEW.vocab.stoi) # 생성된 단어 집합 내 단어 확인
print(LABEL.vocab.stoi) # 생성된 단어 집합 내 단어 확인

단어 집합의 크기 : 44244
defaultdict(<bound method Vocab._default_unk_index of <torchtext.legacy.vocab.Vocab object at 0x7fa9c496eb90>>, {'<unk>': 0, '<pad>': 1, 'the': 2, 'trump': 3, 'to': 4, 'a': 5, 'of': 6, 'and': 7, 'in': 8, 'that': 9, 'his': 10, 'for': 11, 'on': 12, 'he': 13, 'is': 14, 'with': 15, 'donald': 16, 'said': 17, 'as': 18, 'president': 19, 'has': 20, 'was': 21, 'at': 22, 'by': 23, 'it': 24, 'who': 25, 'from': 26, 'not': 27, 'be': 28, 'but': 29, '<PAD>': 30, 'an': 31, 'have': 32, 'about': 33, 'would': 34, 'had': 35, 'will': 36, 'campaign': 37, 'i': 38, 'this': 39, 'are': 40, 'full': 41, 'or': 42, 'him': 43, 'after': 44, 'text': 45, 'new': 46, 'they': 47, 'if': 48, 'one': 49, 'when': 50, 'more': 51, 'house': 52, 'clinton': 53, 'republican': 54, 'its': 55, 'been': 56, 'out': 57, 'what': 58, 'their': 59, 'which': 60, 'us': 61, 'white': 62, 'up': 63, 'her': 64, 'also': 65, 'were': 66, 'than': 67, 'administration': 68, 'people': 69, 'last': 70, 'over': 71, 'all': 72, 'no': 73, 'could

In [39]:
BATCH_SIZE = 128

# BucketIterator : 모든 텍스트 작업을 일괄로 처리하고 단어를 인덱스 숫자로 변환 하는것을 도움 
train_loader, valid_loader, test_loader = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.REVIEW), # 길이가 유사한 것을 일괄 처리하고, 패딩을 최소화하기위해 길이로 정렬
    sort_within_batch = True) # 내림차순 정렬

In [10]:
# Model 
class CNN_classifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_kernels, kernel_sizes, output_dim, dropout, pad_idx):
        super(CNN_classifier, self).__init__()
        self.embedding = nn.Embedding(num_embeddings = vocab_size, # 임베딩을 할 단어들의 개수 (단어 집합의 크기)
                                      embedding_dim = embedding_dim, # 임베딩 할 벡터의 차원 (하이퍼파라미터)
                                      padding_idx = pad_idx) # 패딩을 위한 토큰의 인덱스
        self.convs = nn.ModuleList([nn.Conv2d(in_channels = 1, # input channel수 ( ex RGB 이미지 = 3 )
                                              out_channels = n_kernels, # convolution에 의해 생성될 channel의 수
                                              kernel_size = (ksize, embedding_dim)) # ksize만 변화. embedding_dim은 고정
                                    for ksize in kernel_sizes])
        self.fc = nn.Linear(len(kernel_sizes)*n_kernels, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, review):
        embedded = self.embedding(review)
        embedded = embedded.unsqueeze(1) # 특정 위치에 1인 차원을 추가 <-> squeeze : 1인 차원을 제거
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        cat = self.dropout(torch.cat(pooled, dim = 1))
        res = self.fc(cat)
        return res

In [28]:
# Model
class LSTM_classifier(nn.Module):
    def __init__(self, n_layers, hidden_dim, vocab_size, embedding_dim,  output_dim, dropout, pad_idx):
        super(LSTM_classifier, self).__init__()
        self.n_layers = n_layers
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(num_embeddings = vocab_size, # 임베딩을 할 단어들의 개수 (단어 집합의 크기)
                                      embedding_dim = embedding_dim, # 임베딩 할 벡터의 차원 (하이퍼파라미터)
                                      padding_idx = pad_idx) # 패딩을 위한 토큰의 인덱스
        self.lstm = nn.LSTM(self.embedding_dim, self.hidden_dim, num_layers=self.n_layers, dropout=dropout)
        self.fc = nn.Linear(self.hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, review):
        input = self.embedding(review)
        batch_size = review.shape[1]
        h = torch.zeros(1, batch_size, self.hidden_dim).to(device)
        c = torch.zeros(1, batch_size, self.hidden_dim).to(device)
        out, (h, c) = self.lstm(input, (h, c))
        out = self.dropout(out)
        out = self.fc(out[:, -1, :])

        return out

In [49]:
# 모델 선언
INPUT_DIM = len(REVIEW.vocab)
EMBEDDING_DIM = 300
N_KERNELS = 100
KERNEL_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = REVIEW.vocab.stoi[REVIEW.pad_token]
N_LAYERS = 1
HIDDEN_DIM = 512
LEARNING_RATE = 1e-3

model = CNN_classifier(INPUT_DIM, EMBEDDING_DIM, N_KERNELS, KERNEL_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX).to(device)
# model = LSTM_classifier(N_LAYERS, HIDDEN_DIM, INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, DROPOUT, PAD_IDX).to(device)

print('모델 파라미터 수 :', sum(param.numel() for param in model.parameters() if param.requires_grad))

모델 파라미터 수 : 13633801


In [40]:
# 사전 훈련된 단어 벡터 불러오기
pretrained_weight = REVIEW.vocab.vectors
print(pretrained_weight.shape, model.embedding.weight.data.shape)
print(model.embedding.weight.data.copy_(pretrained_weight))
UNK_IDX = REVIEW.vocab.stoi[REVIEW.unk_token]

# unk, pad token -> 0 처리 
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

torch.Size([44243, 300]) torch.Size([44243, 300])
tensor([[ 0.3374, -0.1778, -0.3035,  ..., -0.2523,  1.0669, -0.2985],
        [ 0.2574,  0.6934, -0.1463,  ..., -0.2464, -0.6138, -0.3650],
        [ 0.0104, -0.1829,  0.0761,  ..., -0.1362, -0.2240, -0.0552],
        ...,
        [-0.1918, -0.8905, -0.8683,  ..., -1.6791, -0.3633,  0.1789],
        [-0.6007, -0.5074, -1.4654,  ...,  1.0839,  1.4596,  0.5614],
        [ 1.7275,  0.1482,  1.7116,  ..., -1.1709,  0.2279,  0.0791]],
       device='cuda:0')


In [41]:
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.BCEWithLogitsLoss() # BCELoss + sigmoid

In [15]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds==y).float()
    acc = correct.sum() / len(correct)
    return acc

In [23]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        X, y = batch.REVIEW.to(device), batch.LABEL.to(device)
        optimizer.zero_grad()
        predictions = model(X).squeeze(1) # output_dim = 1
        loss = criterion(predictions, y)
        acc = binary_accuracy(predictions, y)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

In [22]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            X, y = batch.REVIEW.to(device), batch.LABEL.to(device)
            predictions = model(X).squeeze(1)
            loss = criterion(predictions, y)
            acc = binary_accuracy(predictions, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [50]:
N_EPOCHS = 20
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_loader, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'Classifier.pt') # 모델 저장
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

RuntimeError: ignored

In [45]:
model.load_state_dict(torch.load('Classifier.pt'))

test_loss, test_acc = evaluate(model, test_loader, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.689 | Test Acc: 53.23%
